In [1]:
import rosbag
import rospy
from ninebot_gio_coupling.msg import GnssEphemMsg, GnssGloEphemMsg, GnssMeasMsg, GnssObsMsg, GnssTimeMsg, StampedFloat64Array
from nav_msgs.msg import Odometry
import time
import subprocess
import pandas as pd
roscore = subprocess.Popen('roscore')
time.sleep(1)  # wait a bit to be sure the roscore is really launched

# out_bag_name = '/persist/source_data1/wangyf/dataset/GNSS/GVINS/sports_inoutdoor.bag'
# out_bag_name = '/persist/source_data1/wangyf/dataset/GNSS/GVINS/sports_vins.bag'
out_bag_name = '/persist/source_data1/wangyf/dataset/GNSS/GVINS/sports_urban_driving.bag'
file_name = "/persist/catkin_ws/vins_output_sport_field/vins_result_no_loop.csv"
# file_name = "/persist/catkin_ws/dataset/gvins_analyse/inoutdoor/vins.csv"
rospy.init_node('vins2bag')
bag = rosbag.Bag(out_bag_name, 'w')
vins_df = pd.read_csv(file_name, sep=',', header=None)
print(vins_df.head)
def generate_odom_msg(timestamp, px, py, pz, qw, qx, qy, qz, vx, vy, vz):
    #print(timestamp, px, py, pz, qw, qx, qy, qz, vx, vy, vz)
    odom_msg = Odometry()
    odom_msg.header.stamp.secs = int(timestamp / 1e9)
    odom_msg.header.stamp.nsecs = int(timestamp % 1e9)
    odom_msg.header.frame_id = "vins"
    odom_msg.pose.pose.position.x = px
    odom_msg.pose.pose.position.y = py
    odom_msg.pose.pose.position.z = pz
    odom_msg.pose.pose.orientation.w = qw
    odom_msg.pose.pose.orientation.x = qx
    odom_msg.pose.pose.orientation.y = qy
    odom_msg.pose.pose.orientation.z = qz
    odom_msg.twist.twist.linear.x = vx
    odom_msg.twist.twist.linear.y = vy
    odom_msg.twist.twist.linear.z = vz
    return odom_msg

msgs = vins_df.apply(lambda x: generate_odom_msg(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10]), axis=1)

for msg in msgs:
    bag.write('vins_odom', msg, t=msg.header.stamp)
bag.close()
    

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


<bound method NDFrame.head of                         0         1         2         3        4        5   \
0      1606691895448146688  -0.02319   0.07241  -0.00524  0.70028 -0.70821   
1      1606691895548146432  -0.02788   0.11928  -0.01498  0.69560 -0.71211   
2      1606691895648146432  -0.02778   0.14301  -0.01498  0.69358 -0.71383   
3      1606691895748146432  -0.02988   0.17443  -0.01443  0.69337 -0.71368   
4      1606691895848146432  -0.02720   0.20911  -0.01654  0.69193 -0.71443   
...                    ...       ...       ...       ...      ...      ...   
14196  1606693315048146432 -68.60749 -60.79121 -39.60032  0.36958 -0.40236   
14197  1606693315148146432 -68.46705 -60.83966 -39.61716  0.37172 -0.40681   
14198  1606693315248146432 -68.33110 -60.88336 -39.63843  0.37201 -0.40262   
14199  1606693315348146432 -68.18752 -60.93472 -39.63590  0.36194 -0.39617   
14200  1606693315448146688 -68.06315 -60.98060 -39.62055  0.32981 -0.41165   

            6        7        8  

In [1]:
import rosbag
import rospy
from ninebot_gio_coupling.msg import GnssEphemMsg, GnssGloEphemMsg, GnssMeasMsg, GnssObsMsg, GnssTimeMsg, StampedFloat64Array, GnssPolyEphemMsg, GnssPreciseEphemMsg
from nav_msgs.msg import Odometry
import sys
sys.path.append('../')
from laika.raw_gnss import normal_meas_from_array, process_measurements

import numpy as np
from pathlib import Path
from laika import AstroDog
from tqdm.auto import tqdm
import time
import subprocess
import pandas as pd
import os
dog = AstroDog(dgps=True)

TypeError: findCaller() takes from 1 to 2 positional arguments but 3 were given

In [7]:
# roscore = subprocess.Popen('roscore')
# time.sleep(1)  # wait a bit to be sure the roscore is really launched
import sys
sys.path.append('../')
from laika.raw_gnss import normal_meas_from_array, process_measurements

import numpy as np
from pathlib import Path
from laika import AstroDog
from tqdm.auto import tqdm
dog = AstroDog(dgps=True)
dir_name = '/persist/source_data1/wangyf/dataset/GNSS/comma2k19/Chunk_1/b0c9d2329ad1606b|2018-07-27--06-03-57/3/processed_log/GNSS/raw_gnss_ublox'
out_bag_name = os.path.join(dir_name, 'gnss_data.bag')
base_dir = Path(dir_name)
raw_ublox_t = np.load(base_dir / 't')
raw_ublox = np.load(base_dir / 'value')
measurements = np.array([normal_meas_from_array(arr) for arr in raw_ublox])
grouped_t = np.unique(raw_ublox_t)
grouped_meas_processed = []
# process measurement groups
for t in tqdm(grouped_t):
    meas = measurements[raw_ublox_t == t]
    # print("1",type(meas[-1].sat_ephemeris), meas[-1].sat_ephemeris, meas[-1].sat_ephemeris.prn)
    grouped_meas_processed.append(process_measurements(meas, dog))

  0%|          | 0/600 [00:00<?, ?it/s]